## TL;DR

- Different ways words are represented by computers
  - WordNet : manual labeling, traditional method
  - WordVectors
    - One-Hot Vectors
    - Word Vectors

## WordNet
- Wordnet is a lexical database of semantic relations between words in English first created by CogSys Lab of Princeton University.
- It includes N, V, ADJ, ADV but omits PREP, DET, and other function words.
- WordVec for other langauges exists too.

### Wordnet Example

helloworld


### Limitations
- Requires human labor
  - Impossible to update every word
- Missing **nuance**
  - "proficient" is listed as a synoynm for "good"
- Misses new words
  - badass, nifty, etc

### WordNet Example

In [8]:
!pip install nltk

In [9]:
from nltk.corpus import wordnet as wn

print('Synsets for the word "invite" in WordNet:\n\n', wn.synsets('invite'))

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/Users/joohunhyun/nltk_data'
    - '/opt/anaconda3/envs/cs224n/nltk_data'
    - '/opt/anaconda3/envs/cs224n/share/nltk_data'
    - '/opt/anaconda3/envs/cs224n/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
